# DataQuest Guided Project 
###  Profitable App Profiles for the App Store and Google Play Markets

This project is meant to help me learn Python skills (general, data cleaning and data analysis) as well as familiarize myself with Jupyter Notebook.  Focus was on readable code and output, and using comprehensions wherever I could in place of for loops.

NOTE: I did not consult the [solution](https://github.com/dataquestio/solutions/blob/master/Mission350Solutions.ipynb) while coding, but did check afterwards to make sure I was getting the same numerical results.

Datasets used are located here:

[Google Play Store Apps](https://www.kaggle.com/lava18/google-play-store-apps/)

[Mobile App Store](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/)


#### First we open and read the dataset files (CSV format) into a list of lists
Store the header and data for each, then display the header fields

In [1]:
from csv import reader

def prepare_datafile(filepath):
    '''open and read CSV files into Dataset, display header fields'''
    try:
        file = open(filepath, encoding="utf-8")
        dataset = list(reader(file))
        header = dataset[0:1][0]
        data = dataset[1:]
        print('Header Fields ({}):\n{}\n'.format(len(header),header))
        file.close
    except FileNotFoundError:
        print('***********{} not found.\n'.format(filepath))
        return [],[]
    return (header, data)

print('Preparing Apple Store data file...')
apple_header, apple_data = prepare_datafile(
    'E:/courses/dataquest/app-store-apple-data-set-10k-apps/AppleStore.csv')

print('Preparing Google Play data file...')
google_header, google_data = prepare_datafile(
    'E:/courses/dataquest/google-play-store-apps/googleplaystore.csv')


Preparing Apple Store data file...
Header Fields (17):
['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']

Preparing Google Play data file...
Header Fields (13):
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']



#### Get a first look at the data, including row and column counts.  
Also, display a few sample rows

In [2]:
# Constants for commonly used values
GOOGLE_NAME_IDX=0
APPLE_NAME_IDX=2

def explore_data(data, start, end):
    '''function for viewing data'''
    print('# of rows: {:,}\n# of columns: {}\n\nExamples:'
          .format(len(data),len(data[0])))
    #List comprehension to print out every row in slice
    [print(row) for row in data[start:end]]
    print('\n')
            
# First look at the data
print("Apple Store Data:")
explore_data(data=apple_data, start=0, end=3)

print("Google Play Data:")
explore_data(data=google_data, start=0, end=3)

Apple Store Data:
# of rows: 7,197
# of columns: 17

Examples:
['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']
['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']
['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3.5', '4.5', '5.0.0', '4+', 'Weather', '37', '5', '3', '1']


Google Play Data:
# of rows: 10,841
# of columns: 13

Examples:
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']
['U Launcher Lite – FREE Live Cool Themes, H

# Data Cleaning

### First, we check for corrupt records and delete them.

In [3]:
#Data cleaning
def remove_corrupt_records(data, header_len):
    '''find problem row/s (# of fields not equal to header fields) and remove them'''
    for row in data:
        if len(row) != header_len:
            idx = data.index(row)
            print('**** Deleting corrupt record at index {}:\n{}'.format(idx,row))
            del data[idx]
            
# First, Android Apps
print('\nChecking for corrupt row/s in Google Play data...\n')
remove_corrupt_records(data=google_data, header_len=len(google_header))

# Now, Apple Apps
print('\nChecking for corrupt row/s in Apple Store data...\n')
remove_corrupt_records(data=apple_data, header_len=len(apple_header))


Checking for corrupt row/s in Google Play data...

**** Deleting corrupt record at index 10472:
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']

Checking for corrupt row/s in Apple Store data...



### Next, we check for duplicate records, combining them into a single record for the app, based on entry with the most recent review count

In [4]:

def find_duplicates(data, name_idx):
    '''find duplicate app records'''
    unique_apps = []
    duplicate_apps = []
    for row in data:
        name = row[name_idx]
        if(name in unique_apps):
            duplicate_apps.append(name)
        else:
            unique_apps.append(name)
    
    return duplicate_apps

def combine_duplicates(data, name_idx, n_reviews_idx, dupes):
    '''combine duplicates into single record for app, based on most recent view count'''
    reviews_max = {}
    for app in data:
        name = app[name_idx]
        n_reviews = float(app[n_reviews_idx])
        if name not in reviews_max or reviews_max[name] < n_reviews:
            reviews_max[name] = n_reviews

    # verify correct length of combined duplicates
    if len(reviews_max) != (len(data) - len(dupes)):        
        print("****ERROR: Data length not matching")
        
    # Create new dataset to hold only unique app names
    clean = []
    already_added = []
    
    for app in data:
        name = app[name_idx]
        n_reviews = float(app[n_reviews_idx])
        if n_reviews == reviews_max[name] and name not in already_added:
            clean.append(app)
            already_added.append(name)
    
    if len(clean) != (len(data) - len(dupes)):        
        print("****ERROR: Data length not matching")
    else:
        print('Duplicates Combined!\n\t\tOriginal Data Set Size: {}\n\t\t# of Duplicates:{}\n\t\tClean Data Set Size:{}\n'
              .format(len(data),len(dupes), len(clean)))
        
    return clean

google_dupes = find_duplicates(data=google_data, name_idx=GOOGLE_NAME_IDX)
print('Duplicates in Google Data ({}):'.format(len(google_dupes)))
print(str(google_dupes[:5]), '\n')
print("Combining Duplicates for Google Data...")
google_clean = combine_duplicates(
    data=google_data, name_idx=GOOGLE_NAME_IDX, n_reviews_idx=3, dupes=google_dupes)

apple_dupes = find_duplicates(data=apple_data, name_idx=APPLE_NAME_IDX)
print('Duplicates in Apple Data ({}):'.format(len(apple_dupes)))
print(str(apple_dupes[:5]), '\n') 
print("Combining Duplicates for Apple Data...")
apple_clean = combine_duplicates(
    data=apple_data, name_idx=APPLE_NAME_IDX, n_reviews_idx=6, dupes=apple_dupes)

Duplicates in Google Data (1181):
['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings'] 

Combining Duplicates for Google Data...
Duplicates Combined!
		Original Data Set Size: 10840
		# of Duplicates:1181
		Clean Data Set Size:9659

Duplicates in Apple Data (2):
['VR Roller Coaster', 'Mannequin Challenge'] 

Combining Duplicates for Apple Data...
Duplicates Combined!
		Original Data Set Size: 7197
		# of Duplicates:2
		Clean Data Set Size:7195



### Remove apps with non-English names

In [5]:
def is_english(str):
    '''determine if string contains no more than 3 non-english characters'''
    non_english_chars = 0
    for char in str:
        if ord(char) > 127:
            non_english_chars += 1
    
    return non_english_chars <= 3

def remove_non_english_apps(data, name_idx):
    '''removes non-english apps from a dataset'''
    #list comprehension to include all english apps
    return [app for app in data if is_english(app[name_idx])]

def print_non_english(clean, english, name_idx):
    '''print all the non-english app names'''
    # set comprehensions to grab just the name values
    clean_set = {app[name_idx] for app in clean}
    english_set = {app[name_idx] for app in english}  
    # update clean set to be all the names that are in clean, but not in english
    clean_set -= english_set        
    print('***********\n{} Non-english App names removed from data:\n'.format(len(clean)-len(english)))
    {print(name) for name in clean_set} 
    print('***********\nBefore: {}\nAfter: {}\n'.format(len(clean), len(english)))

# TEST
#print(is_english('Instagram'))
#print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
#print(is_english('Docs To Go™ Free Office Suite'))
#print(is_english('Instachat 😜'))

print('Removing non-English apps from Google Play data...\n')
google_clean_english = remove_non_english_apps(data=google_clean, name_idx=GOOGLE_NAME_IDX)
print_non_english(clean=google_clean, english=google_clean_english, name_idx=GOOGLE_NAME_IDX)
 
print('Removing non-English apps from Apple Store data...\n')    
apple_clean_english = remove_non_english_apps(data=apple_clean, name_idx=APPLE_NAME_IDX)
print_non_english(clean=apple_clean, english=apple_clean_english, name_idx=APPLE_NAME_IDX)


Removing non-English apps from Google Play data...

***********
45 Non-english App names removed from data:

বাংলাflix
감성학원 BL 첫사랑
BL 女性向け恋愛ゲーム◆俺プリクロス
FP Разбитый дисплей
FAHREDDİN er-RÂZİ TEFSİRİ
বাংলা টিভি প্রো BD Bangla TV
RPG ブレイジング ソウルズ アクセレイト
哈哈姆特不EY
Offline Jízdní řády CG Transit
Cъновник BG
BL 女性向け恋愛ゲーム◆ごくメン
Билеты ПДД CD 2019 PRO
DG ग्राम / Digital Gram Panchayat
أحداث وحقائق | خبر عاجل في اخبار العالم
الفاتحون Conquerors
EG SIM CARD (EGSIMCARD, 이지심카드)
英漢字典 EC Dictionary
中国語 AQリスニング
Bonjour 2017 Abidjan CI ❤❤❤❤❤
RMEduS - 음성인식을 활용한 R 프로그래밍 실습 시스템
L.POINT - 엘포인트 [ 포인트, 멤버십, 적립, 사용, 모바일 카드, 쿠폰, 롯데]
CK 初一 十五
DM הפקות
I'm Rich/Eu sou Rico/أنا غني/我很有錢
РИА Новости
ECナビ×シュフー
Ey Sey Storytime រឿងនិទានតាឥសី
日本AV历史
AJ렌터카 법인 카셰어링
Vip视频免费看-BT磁力搜索
DW فارسی By dw-arab.com
صور حرف H
BQ-መጽሐፍ ቅዱሳዊ ጥያቄዎች
Al Quran Free - القرآن (Islam)
뽕티비 - 개인방송, 인터넷방송, BJ방송
သိင်္ Astrology - Min Thein Kha BayDin
AÖF Ev İdaresi 1. Sınıf
لعبة تقدر تربح DZ
あなカレ【BL】無料ゲーム
パーリーゲイツ公式通販｜EJ STYLE（イージェイスタイル）
BS Calendar

零基础学音标
内涵段子 - 精选搞笑图片、视频
JR東日本アプリ
熱血高校！胴上げ部
开挂江湖-Q版武侠挂机
RPG 偽りの物語 / ドット絵ロールプレイングゲーム
英熟語ターゲット1000（4訂版）
闲鱼-让你的闲置游起来
尚品网 - 全球精品高端时尚购物网站
兜町アナリストがお伝えする「兜予報」（無料）
一起捕鱼 - 欢乐街机电玩城
Fit 私人健身教练-超有效的健康运动减肥软件
ねこげーむ~ねこ騎士ポチ~簡単で人気の猫ゲーム~基本無料
楽天カード
驴妈妈旅游-订景点门票机票火车票特价酒店
鴨川等間隔の法則
頭の回転をはやくする！脳トレ！Blackhole
土豆视频HD—高清影视综艺视频播放器
360手机浏览器-安全上网看新闻、安心搜索、极速抢票
戦艦帝国-200艘の実在戦艦を集めろ (2周年記念&世界2000万DL)
ひとりぼっち惑星
英雄Q传-穿越回合制卡牌手游，梦幻般的剧情，开启3D热血传奇人生
pixivコミック - みんなのマンガアプリ
58同城-招聘找工作兼职租房软件
激ムズ！ねこじゃんぷ２
Clash of Richers 3 （城市富翁3）
デビルサバイバー２ 最後の７日間
脱出ゲーム - Solid - 無機質な部屋からの 脱出
国金宝理财-15%年化收益的金融投资赚钱软件
全民夺宝(官方)
赤ペン 提出カメラ
セブン‐イレブンアプリ
贝贝-8000万妈妈信赖的母婴正品特卖拼团商城
病みカワカメラ
全民打金币——爽快的正版放置手游
猫眼电影 - 一网打尽好电影
ゲッターズ飯田の占い
【ひっぱりパズル】ズキュ〜〜〜〜ン！
花椒直播-高清美颜直播互动平台
秒拍-超火爆短视频分享平台
意味が分かると怖い話-この怖い話の意味分かる？【意味怖】
クリスマスミッケ/脱出ゲーム感覚の絵探しパズルゲーム
ATOK -日本語入力キーボード
妹型杀器
【ドレ？ドコ？】脱出ゲーム感覚の謎解きパズルゲーム
网易新闻 - 精选好内容，算出你的兴趣
艶が～るプレミアム 女性向け恋愛ゲーム！乙女げーむ
【明星恋爱】偶像之路TIME TO STAR
ペットとあそぼ！- 声を選んで写真がしゃべる、しりとりゲームおしゃべりペット！犬・猫好き癒し系アプリ
携程旅行-酒店、机票、火车票预订助手
蘑菇街-网红直播搭配的购物特卖平台
大空ヘクタール農園
TOEIC®テスト文法640問1


### Remove non-Free apps from Data

In [6]:
def remove_non_free_apps(data, price_idx):
    '''remove all non-free apps from dataset'''
    #list comprehension to include all free apps
    free_apps = [app for app in data if float(app[price_idx].replace('$', '')) == 0]
    print('Removed {} non-free apps.\n{} apps in data set [free/clean/english]\n'
          .format(len(data) - len(free_apps),len(free_apps)))
    return free_apps


print('Removing non-free apps from Apple Store data...')
apple_clean_english_free = remove_non_free_apps(data=apple_clean_english, price_idx=5)

print('Removing non-free apps from Google Play data...')
google_clean_english_free = remove_non_free_apps(data=google_clean_english, price_idx=7)

Removing non-free apps from Apple Store data...
Removed 2961 non-free apps.
3220 apps in data set [free/clean/english]

Removing non-free apps from Google Play data...
Removed 750 non-free apps.
8864 apps in data set [free/clean/english]



# Data Analysis

#### Searching for Apps that are successful in both Google and Apple stores

The aim of the data analysis is to find app profiles that are successful in both markets.

First we look at categories and genre, broken down by percentage

In [7]:
# replace longer variable names
google_data = google_clean_english_free
apple_data = apple_clean_english_free
       
def freq_table(data, index):
    '''construct frequency table from dataset, for values at specified index'''
    table = {}
    total = len(data)
    for app in data:
        value = app[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    # use dict comprehension to transform to %age of total
    return {key : (table[key]/total) * 100 for key in table}
    
    
def display_table(table, round_num=True):
    '''display table as reverse-sorted data'''
    #This function was provided as part of guided project
    #Enhanced to round numbers by default, but also allow for formatting numbers w/ commas
    table_display = []
    for key in table:
        # change to tuple so value can be sorted below
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        if round_num:
            print(entry[1], ':', round(float(entry[0]),2))
        else:            
            print(entry[1], ':', '{:,}'.format(int(entry[0])))
    
    
# Display sorted frequency table for 'prime_genre' field in Apple Data
print('\nDisplay Frequency Table of Apple Store\'s top genres by percentage:\n')
display_table(table=freq_table(data=apple_data, index=12))
# Display sorted frequency table for 'Category' field in Google Data
print('\nDisplay Frequency Table of Google Play\'s top categories by percentage:\n')
display_table(table=freq_table(data=google_data, index=1))
# Display sorted frequency table for 'Genre' field in Google Data
print('\nDisplay Frequency Table of Google Play\'s top genres by percentage:\n')
display_table(table=freq_table(data=google_data, index=9))


Display Frequency Table of Apple Store's top genres by percentage:

Games : 58.14
Entertainment : 7.89
Photo & Video : 4.97
Education : 3.66
Social Networking : 3.29
Shopping : 2.61
Utilities : 2.52
Sports : 2.14
Music : 2.05
Health & Fitness : 2.02
Productivity : 1.74
Lifestyle : 1.58
News : 1.34
Travel : 1.24
Finance : 1.12
Weather : 0.87
Food & Drink : 0.81
Reference : 0.56
Business : 0.53
Book : 0.43
Navigation : 0.19
Medical : 0.19
Catalogs : 0.12

Display Frequency Table of Google Play's top categories by percentage:

FAMILY : 18.91
GAME : 9.72
TOOLS : 8.46
BUSINESS : 4.59
LIFESTYLE : 3.9
PRODUCTIVITY : 3.89
FINANCE : 3.7
MEDICAL : 3.53
SPORTS : 3.4
PERSONALIZATION : 3.32
COMMUNICATION : 3.24
HEALTH_AND_FITNESS : 3.08
PHOTOGRAPHY : 2.94
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.66
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.25
BOOKS_AND_REFERENCE : 2.14
DATING : 1.86
VIDEO_PLAYERS : 1.79
MAPS_AND_NAVIGATION : 1.4
FOOD_AND_DRINK : 1.24
EDUCATION : 1.16
ENTERTAINMENT : 0.96
LIBRARIES_AND_DEMO 

Average user rating per genre.

In [8]:
def get_average_for_genre(data, genre_idx, value_idx):
    '''find averages for specified values per genre'''
    avg_table = {}
    genres_table = freq_table(data, genre_idx)
    for genre in genres_table:        
        #comprehension to grab only apps that match current genre
        genre_apps = [app for app in data if app[genre_idx] == genre]
        total = 0
        n_genre = 0
        # loop through all apps for this genre, obtaining a count and total value
        for app in genre_apps:
            n_genre += 1
            # avoid NaN values which will throw off total
            if app[value_idx] != 'NaN':
                # remove non-numeric characters if any before passing to float
                total += float(app[value_idx].replace('+','').replace(',','').replace('$',''))
        #add genre to table
        avg_table[genre] = total/n_genre
        
    return avg_table
        
print('\nDisplaying Average Genre Ratings for Apple Data...\n')
display_table(table=get_average_for_genre(apple_data, 12, 8))

print('\nDisplaying Average Genre Ratings for Google Data...\n')
display_table(table=get_average_for_genre(google_data, 9, 2))


Displaying Average Genre Ratings for Apple Data...

Catalogs : 4.12
Games : 4.04
Productivity : 4.0
Business : 3.97
Shopping : 3.97
Music : 3.95
Photo & Video : 3.9
Navigation : 3.83
Health & Fitness : 3.77
Reference : 3.67
Education : 3.64
Food & Drink : 3.63
Social Networking : 3.59
Entertainment : 3.54
Utilities : 3.53
Travel : 3.49
Weather : 3.48
Lifestyle : 3.41
Finance : 3.38
News : 3.24
Book : 3.07
Sports : 3.07
Medical : 3.0

Displaying Average Genre Ratings for Google Data...

Comics;Creativity : 4.8
Health & Fitness;Education : 4.7
Strategy;Action & Adventure : 4.6
Puzzle;Education : 4.6
Simulation;Pretend Play : 4.55
Entertainment;Creativity : 4.53
Tools;Education : 4.5
Strategy;Education : 4.5
Sports;Action & Adventure : 4.5
Racing;Pretend Play : 4.5
Arcade;Pretend Play : 4.5
Casual;Brain Games : 4.47
Music;Music & Video : 4.45
Education;Brain Games : 4.43
Strategy;Creativity : 4.4
Simulation;Education : 4.4
Puzzle;Creativity : 4.4
Entertainment;Education : 4.4
Adventure;A

Average User totals per genre

In [9]:
print('\nDisplaying Average User totals for Apple Data...\n')
display_table(table=get_average_for_genre(
    data=apple_data, genre_idx=12, value_idx=6), round_num=False)

# This data is a bit off as 'Installs' column contain inexact values (100+, 1,000+, 100,000+, etc)
print('\nDisplaying Average User totals for Google Data...\n')
display_table(table=get_average_for_genre(
    data=google_data, genre_idx=9, value_idx=5), round_num=False)


Displaying Average User totals for Apple Data...

Navigation : 86,090
Reference : 74,942
Social Networking : 71,548
Music : 57,326
Weather : 52,279
Book : 39,758
Food & Drink : 33,333
Finance : 31,467
Photo & Video : 28,441
Travel : 28,243
Shopping : 26,919
Health & Fitness : 23,298
Sports : 23,008
Games : 22,812
News : 21,248
Productivity : 21,028
Utilities : 18,684
Lifestyle : 16,485
Entertainment : 14,029
Business : 7,491
Education : 7,003
Catalogs : 4,004
Medical : 612

Displaying Average User totals for Google Data...

Communication : 38,456,119
Adventure;Action & Adventure : 35,333,333
Video Players & Editors : 24,947,335
Social : 23,253,652
Arcade : 22,888,365
Casual : 19,569,221
Puzzle;Action & Adventure : 18,366,666
Photography : 17,840,110
Educational;Action & Adventure : 17,016,666
Productivity : 16,787,331
Racing : 15,910,645
Travel & Local : 14,051,476
Casual;Action & Adventure : 12,916,666
Action : 12,603,588
Strategy : 11,199,902
Tools : 10,802,461
Tools;Education : 10,